#Install dependencies

In [ ]:
!pip install tensorflow-addons
!pip install transformers
!pip install --upgrade gensim==3.8.3
!pip install optuna
!pip install -q tf-models-official==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.6 MB/s 
     |████████████████████████████████| 596 kB 18.0 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.2 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-p

# Import libraries. Mount corpora

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import pickle as pk
import pandas as pd
import numpy as np
import optuna
import random, os

from transformers import TFAutoModel, AutoTokenizer
from google.colab import drive
from pathlib import Path


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from transformers import AutoModel, AutoTokenizer, TFAutoModel

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
FB_RO_OFFENSE_PATH = Path('/content/drive/MyDrive/Licenta_Busuioc_Gabriel-Razvan_2022/corpora/FB-RO-Offense')
RO_OFFENSE_PATH = Path('/content/drive/MyDrive/Licenta_Busuioc_Gabriel-Razvan_2022/corpora/RO-Offense')

df_trainRO = pd.read_csv(RO_OFFENSE_PATH / 'train_internal.csv')
df_testRO = pd.read_csv(RO_OFFENSE_PATH / 'test.csv')
df_validateRO = pd.read_csv(RO_OFFENSE_PATH / 'validation_internal.csv')
df_trainFBRO = pd.read_csv(FB_RO_OFFENSE_PATH / 'train_internal.csv')
df_testFBRO = pd.read_csv(FB_RO_OFFENSE_PATH / 'test.csv')
df_validateFBRO = pd.read_csv(FB_RO_OFFENSE_PATH / 'validation_internal.csv')

len(df_trainRO), len(df_validateRO), len(df_testRO), len(df_trainFBRO), len(df_validateFBRO), len(df_testFBRO)

(7962, 1991, 2492, 2851, 713, 891)

In [ ]:

labelCodificationMap = {"OTHER":0, "PROFANITY":1, "INSULT":2, "ABUSE":3}

x_trainRO, y_trainRO = df_trainRO.text, df_trainRO.label.map(labelCodificationMap)
x_validateRO, y_validateRO = df_validateRO.text, df_validateRO.label.map(labelCodificationMap)
x_testRO, y_testRO = df_testRO.text, df_testRO.label.map(labelCodificationMap)

x_trainFBRO, y_trainFBRO = df_trainFBRO.text, df_trainFBRO.label.map(labelCodificationMap)
x_validateFBRO, y_validateFBRO = df_validateFBRO.text, df_validateFBRO.label.map(labelCodificationMap)
x_testFBRO, y_testFBRO = df_testFBRO.text, df_testFBRO.label.map(labelCodificationMap)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")
bert = TFAutoModel.from_pretrained("readerbench/RoBERT-base")

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/468 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/554M [00:00<?, ?B/s]

Some layers from the model checkpoint at readerbench/RoBERT-base were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at readerbench/RoBERT-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
allData = tokenizer(x_trainRO.tolist() + x_validateRO.tolist() + x_testRO.tolist() + x_trainFBRO.tolist(), return_tensors="np")
allData

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


{'input_ids': array([list([3, 10023, 294, 16444, 18858, 294, 5799, 22, 19322, 288, 7185, 288, 294, 22425, 16347, 9, 1605, 1605, 1605, 46, 465, 29962, 20899, 3974, 13, 4738, 1605, 45, 77, 7176, 7809, 32, 4476, 6, 407, 1248, 6244, 6954, 14, 12506, 5363, 31, 53, 1277, 1605, 4]),
       list([3, 452, 6, 300, 564, 513, 513, 513, 513, 13714, 2155, 6, 7341, 102, 294, 375, 182, 45, 8083, 15, 513, 513, 513, 513, 53, 18240, 2, 13, 473, 513, 513, 513, 513, 19322, 54, 21745, 16780, 513, 513, 513, 11954, 4318, 6, 32398, 9, 513, 513, 513, 3765, 1898, 294, 3765, 4422, 513, 513, 513, 4]),
       list([3, 77, 1299, 2292, 513, 334, 513, 1586, 21, 8551, 19, 29, 28, 31409, 31, 28, 21053, 318, 19, 2083, 160, 14, 6, 5881, 148, 1350, 95, 9305, 147, 77, 25864, 13, 6448, 12305, 74, 9, 77, 1340, 1063, 38, 4270, 72, 35, 16528, 7345, 11, 199, 45, 77, 734, 11, 3438, 52, 2479, 35, 13714, 3155, 3548, 6, 11, 4129, 13, 6397, 62, 61, 11079, 77, 24, 26, 1340, 11, 22, 232, 4]),
       ..., list([3, 780, 1567, 23551, 9423

In [ ]:
sentencesTokensLen = []
for inp in allData['input_ids']:
  sentencesTokensLen.append(len(inp))
st_dev = np.std(sentencesTokensLen)
mean = np.mean(sentencesTokensLen)

max_len_tokenizer = int(mean) + int(st_dev)
st_dev, mean, max_len_tokenizer 

(38.38813132893974, 54.74372384937239, 92)

In [ ]:
Y_trainTRANSF = None
Y_validateTRANSF = None
Y_testTRANSF = None
X_testTRANSF = None
X_trainTRANSF  = None
X_validateTRANSF  = None

In [ ]:
def buildModel(denseSize, tokenizerMaxLen):

  global Y_trainTRANSF
  global Y_validateTRANSF
  global Y_testTRANSF
  global X_testTRANSF
  global X_trainTRANSF 
  global X_validateTRANSF 

  X_trainTRANSF = tokenizer(x_trainRO.tolist() + x_validateRO.tolist() + x_testRO.tolist() + x_trainFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_validateTRANSF = tokenizer(x_validateFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)
  X_testTRANSF = tokenizer(x_testFBRO.tolist(), padding='max_length', return_tensors="np", truncation=True, max_length=tokenizerMaxLen)

  Y_trainTRANSF = tf.one_hot(pd.concat([y_trainRO, y_validateRO, y_testRO, y_trainFBRO]), depth=4)
  Y_validateTRANSF = tf.one_hot(y_validateFBRO, depth=4)
  Y_testTRANSF = tf.one_hot(y_testFBRO, depth=4)

  X_testTRANSF = list(X_testTRANSF.values())
  X_trainTRANSF = list(X_trainTRANSF.values())
  X_validateTRANSF = list(X_validateTRANSF.values())

  token_ids = tf.keras.layers.Input((None, ), dtype=np.int32)
  type_ids = tf.keras.layers.Input((None,), dtype=np.int32)
  attention = tf.keras.layers.Input((None,), dtype=np.int32)
  bert_output = bert(input_ids=token_ids, attention_mask=attention, token_type_ids=type_ids)
  cls_output = bert_output.last_hidden_state[:,0,:]
  # avg_output = tf.keras.layers.GlobalAveragePooling1D()(bert_output.last_hidden_state, mask=attention)
  hidden = tf.keras.layers.Dense(denseSize, activation="tanh")(cls_output)
  output = tf.keras.layers.Dense(4, activation="softmax")(hidden) 
  model = tf.keras.Model(inputs=[token_ids, type_ids, attention], outputs=[output])
  model.layers[3].trainable = False
  # model.summary()


  return model

In [ ]:


def tf_seed(seed=0):
	os.environ['PYTHONHASHSEED'] = str(seed)
	# For working on GPUs from "TensorFlow Determinism"
	os.environ["TF_DETERMINISTIC_OPS"] = str(seed)
	np.random.seed(seed)
	random.seed(seed)
	tf.random.set_seed(seed)

tf_seed(0)

In [ ]:
from sklearn import metrics

def getMetrics(dense_size, learning_rate, batch_size, tokenizer_max_len):
  print(dense_size, learning_rate, batch_size, tokenizer_max_len)
  model = buildModel(dense_size, tokenizer_max_len)
  epochs = 30

  f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
  f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=[tf.metrics.CategoricalAccuracy(), tf.metrics.Precision(), tf.metrics.Recall(), f1_metric_micro, f1_metric_macro])
  
  callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=2,
    mode='auto', baseline=None, restore_best_weights=True)
  
  history = model.fit(x=X_trainTRANSF, y=Y_trainTRANSF, validation_data=(X_validateTRANSF, Y_validateTRANSF), epochs=epochs, batch_size=batch_size, callbacks=[callback], verbose=0)
  results = model.evaluate(X_testTRANSF, Y_testTRANSF, verbose=0)

  y_predTRANSF = model.predict(X_testTRANSF)
  y_predCG = pd.Series(tf.argmax(y_predTRANSF, 1)).map({0:0, 1:1, 2:1, 3:1})
  Y_testCG = pd.Series(tf.argmax(Y_testTRANSF, 1)).map({0:0, 1:1, 2:1, 3:1})

  CGaccuracy = metrics.accuracy_score(Y_testCG, y_predCG)
  CGprecision = metrics.precision_score(Y_testCG, y_predCG)
  CGrecall = metrics.recall_score(Y_testCG, y_predCG)
  CGf1 = metrics.f1_score(Y_testCG, y_predCG)

  model.save_weights('/content/SIMPLE-BERT_TRANSF_'+ str(format(results[1], ".3f")) + '_' + str(dense_size) + '_' + str(learning_rate) + '_' + str(batch_size) + '/')
  
  return results + [CGaccuracy, CGprecision, CGrecall, CGf1]

In [ ]:

def objective(trial):

    dense_size = trial.suggest_categorical('dense_size', [16, 32, 64, 128])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]) 
    batch_size = trial.suggest_categorical('per_gpu_batch_size', [16, 32, 64, 128])
    tokenizer_max_len = trial.suggest_categorical('tokenizer_max_len', [20, 40, 60, 80, 100, 120, 140])

    metrics = getMetrics(dense_size, learning_rate, batch_size, tokenizer_max_len)

    return metrics
    

study = optuna.create_study(study_name="TRANSFstudy-BERT-SIMPLE", directions=['minimize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize', 'maximize'], storage="sqlite:///TRANSFstudy-BERT-SIMPLE.db")
study.optimize(objective, n_trials=50)

[I 2022-06-06 16:56:24,869] A new study created in RDB with name: TRANSFstudy-BERT-SIMPLE


32 1e-05 32 100


[I 2022-06-06 16:58:07,250] Trial 0 finished with values: [1.2258192300796509, 0.4994388222694397, 0.7258883118629456, 0.16049382090568542, 0.26286762952804565, 0.10732075572013855, 0.6004489337822672, 0.5054945054945055, 0.38440111420612816, 0.4367088607594937] and parameters: {'dense_size': 32, 'learning_rate': 1e-05, 'per_gpu_batch_size': 32, 'tokenizer_max_len': 100}. 


32 5e-05 128 80


In [ ]:
from google.colab import files
files.download('/content/TRANSFstudy-BERT-SIMPLE.db') 

In [ ]:
# study = optuna.load_study(study_name="TRANSFstudy-BERT-SIMPLE", storage="sqlite:///TRANSFstudy-BERT-SIMPLE.db")
# study.get_trials()
# study.get_trials()[0].params

#Rerun model

In [ ]:
weightsPath = '/content/TRANSF_0.760_0_3e-05_16/' #change this
learning_rate = 1e-4

loaded_model = buildModel()
loaded_model.load_weights(weightsPath)
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=tf.metrics.CategoricalAccuracy())

results = loaded_model.evaluate(X_testTRANSF, Y_testTRANSF, verbose=1)
results

#Rebuild certain model


In [ ]:
weightsPath = '' #change this
seed = 0 #change this
batch_size = 32 #change this
learing_rate = 1e-3 # change this
epochs = 30

loaded_model = buildModel()
loaded_model.load_weights(weightsPath)

loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=tf.metrics.CategoricalAccuracy())
  
loaded_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_loss', min_delta=0, patience=3, verbose=2,
  mode='auto', baseline=None, restore_best_weights=True)

history = loaded_model.fit(x=X_trainTRANSF, y=Y_trainTRANSF, validation_data=(X_validateTRANSF, Y_validateTRANSF), epochs=epochs, batch_size=batch_size, callbacks=[loaded_callback], verbose=1)
results = loaded_model.evaluate(X_testTRANSF, Y_testTRANSF, verbose=0)
results